# Quantum Harmonic Oscillator Demonstration

This notebook demonstrates how to use the `QuantumHarmonicOscillator` class in `qho.py`. It starts with a brief overview of the equations and moves rapidly toward using the `QuantumHarmonicOscillator` class to make plots.

Citations:

1. Atkins, P. W. & De Paula, J. Atkins’ Physical chemistry. (W.H. Freeman, 2006). Pages 290-207.
2. [Vibration-Rotation Spectrum of HCl on HyperPhysics](http://hyperphysics.phy-astr.gsu.edu/hbase/molecule/vibrot.html#c2)


In [2]:
import matplotlib.pyplot as plt
from qho import QuantumHarmonicOscillator

### Mathematical Overview

#### Reduced Mass of Diatomic Molecule
Given m_1 and m_2 which are masses in kg of each atom in a diatomic molecule:
$$ m_r = {m_1m_2 \over m_1+m_2} $$

#### Quantum Number
$$ v = 0, 1, 2, ... $$

#### Energy levels
$$ \omega = \left(k \over m_r\right)^{1/2} $$
$$ E_v = \left(v + {1 \over 2} \right) \hbar \omega $$

#### Wavefunction